# TimeSeries preparation for LSTM
This Notebook is embedding the time series Data

In [ ]:
import os
import pyarrow.parquet as pq
import numpy as np
import pandas as pd
import pickle
from sklearn.utils import shuffle
import os
import csv
#import spacy
import multiprocessing
import time
from gensim.models import Word2Vec
from gensim.models import FastText
import pandas as pd
import numpy as np
import pyarrow.parquet as pq
import tensorflow as tf
import matplotlib.pyplot as plt
import statistics 

# Per Patient 

In [ ]:
timeseries_per_patient = pq.read_table('Cohort/Time_Series/all_time_series_woProcedures_timeseries_data_per_patient.parquet').to_pandas()
timeseries_per_patient


In [ ]:
seq_LSTM=[]
for index, row in timeseries_per_patient.iterrows():
    sequence=row['unique_concept']
    seq_LSTM.append(sequence[-100:])
seq_LSTM

In [ ]:
seq_LSTM_pad=tf.keras.preprocessing.sequence.pad_sequences(seq_LSTM, value=200)

In [ ]:
seq_LSTM_pad

In [ ]:
import pickle
with open("Cohort/Time_Series/Timeseries_per_patient_LSTM_Data_raw.txt", "wb") as fp:   #
    pickle.dump(seq_LSTM_pad, fp)

In [ ]:
with open("Cohort/Time_Series/Timeseries_per_patient_LSTM_Data_raw.txt", "rb") as fp:   # Unpickling
    timeseries_per_patient_padded = pickle.load(fp)
#timeseries_per_patient_per_day_padded=timeseries_per_patient_per_day_padded#[:3000]
timeseries_per_patient_padded

In [ ]:
# embedd the data with different word2vec embeddings

# Cbow

In [ ]:
#cbow 
model_dir = "Cohort/Time_Series/"
cbow_model0 = Word2Vec.load(model_dir + "cbow_dim20_win5_mc0.bin")

In [ ]:
#convert int in strings
timeseries_per_patient_padded=timeseries_per_patient_padded.astype(str)
timeseries_per_patient_padded

In [ ]:
# embedd all timeseries that have 100 concepts
min_concepts=[]
timeseries_per_patient_embedded=[]
for n in timeseries_per_patient_padded:
    try:
        t=cbow_model0[n]
        timeseries_per_patient_embedded.append(t.tolist())
        min_concepts.append('yes')
        print('new')
        #print(cbow_model0[n])
    except: 
        min_concepts.append('no')
        print('padded')
len(timeseries_per_patient_embedded)

In [ ]:
min_concepts
mrn_with_min_concept = pq.read_table('Cohort/Time_Series/all_time_series_woProcedures_timeseries_data_per_patient.parquet').to_pandas()
mrn_with_min_concept=mrn_with_min_concept.drop('unique_concept', axis=1)
mrn_with_min_concept['min_concepts']=min_concepts
mrn_with_min_concept=mrn_with_min_concept.loc[mrn_with_min_concept['min_concepts']=='yes']
mrn_with_min_concept.to_parquet('Cohort/Time_Series/time_series_per_patient_mrns.parquet')

In [ ]:
#static dataframe for evaluation 
static_features=pq.read_table('Cohort/Feature_Extraction/ALL_HF_cohort_unsupervised_only_after_onset_HF_ALL_all_any_all_mean_medium_cleaned_wLab.parquet').to_pandas()
static_features

In [ ]:
#mrn_with_min_concept=mrn_with_min_concept.set_index('medical_record_number')
time_series_static=pd.merge(static_features, mrn_with_min_concept, left_index=True, right_index=True)
time_series_static=time_series_static.drop(['min_concepts'],axis=1)
time_series_static

In [ ]:
time_series_static.to_parquet('Cohort/Time_Series/time_series_per_patient_static_features.parquet')

In [ ]:
mrn_with_min_concept

In [ ]:
timeseries_per_patient_embedded[1]

In [ ]:
import pickle
with open("Cohort/Time_Series/Timeseries_per_patient_LSTM_Data_embedded_cbow_dim20_win5_mc0.txt", "wb") as fp:   #
    pickle.dump(timeseries_per_patient_per_day_embedded, fp)

In [ ]:
#skipgram
model_dir = "Cohort/Time_Series/"
skipgram_model0 = Word2Vec.load(model_dir + "skipgram_dim20_win5_mc0.bin")

In [ ]:
# embedd all timeseries that have 100 concepts
timeseries_per_patient_embedded=[]
for n in timeseries_per_patient_per_day_padded:
    try:
        t=skipgram_model0[n]
        timeseries_per_patient_embedded.append(t.tolist())
        print('new')
        #print(cbow_model0[n])
    except: 
        print('padded')
len(timeseries_per_patient_embedded)

In [ ]:
timeseries_per_patient_embedded[1]

In [ ]:
import pickle
with open("Cohort/Time_Series/Timeseries_per_patient_LSTM_Data_embedded_skipgram_dim20_win5_mc0.txt", "wb") as fp:   #
    pickle.dump(timeseries_per_patient_embedded, fp)

# Per Patient Per Day

In [ ]:
timeseries_per_patient_per_day = pq.read_table('Cohort/Time_Series/all_time_series_woProcedures_timeseries_data_per_patient_per_day.parquet').to_pandas()
timeseries_per_patient_per_day


In [ ]:
days_per_patient=timeseries_per_patient_per_day['medical_record_number'].value_counts()

In [ ]:
plt.hist(days_per_patient, bins =1000)
plt.show()

In [ ]:
sum(i < 7 for i in days_per_patient)

In [ ]:
#cbow 
model_dir = "Cohort/Time_Series/"
cbow_model0 = Word2Vec.load(model_dir + "cbow_dim20_win5_mc0.bin")

In [ ]:
from operator import add
from operator import truediv
cohort_array=[]
mrn_in_cohort=[]
for mrn in timeseries_per_patient_per_day.medical_record_number.unique():
    patient_array=[]
    print(mrn)
    #get all available days per Patients
    patient=timeseries_per_patient_per_day.loc[timeseries_per_patient_per_day.medical_record_number==str(mrn)]
    days_count=patient.shape[0]
    if days_count >=7:
        mrn_in_cohort.append(mrn)
        for index, day in patient.iterrows():
            #get concepts per day
            #print(day['unique_concept'])
            seq_day=day['unique_concept']        
            conepts_per_day=len(seq_day)
            #convert int into strings
            seq_day=seq_day.astype(str)
            #embedd the concepts: 
            #emtpy start array: 
            day_sum=[0]*20
            embedded_day=[]
            for concept in seq_day:
                embedded_day=[]
                t=cbow_model0[concept]
                embedded_day.append(t.tolist())
                #sum of embedded concepts
                day_sum=list( map(add, embedded_day[0], day_sum) )
            #average of embedded concepts
            day_sum=list(map(lambda x: x/conepts_per_day, day_sum))
            patient_array.append(day_sum)
        cohort_array.append(patient_array[-7:])
    

In [ ]:
len(mrn_in_cohort)

In [ ]:
conepts_per_day

In [ ]:
embedded_day[0]

In [ ]:
day_sum

In [ ]:
patient_array[-1:]

In [ ]:
cohort_array[0][-2:]

In [ ]:
len(cohort_array[1000][7])

In [ ]:
import pickle
with open("Cohort/Time_Series/Timeseries_per_patient_per_day_LSTM_Data_embedded_cbow_dim20_win5_mc0.txt", "wb") as fp:   #
    pickle.dump(cohort_array, fp)

In [ ]:
with open("Cohort/Time_Series/Timeseries_per_patient_per_day_LSTM_Data_embedded_cbow_dim20_win5_mc0.txt", "rb") as fp:   # Unpickling
    timeseries_per_patient_per_day = pickle.load(fp)
len(timeseries_per_patient_per_day)

In [ ]:
#get static features for per day data 
static_features

In [ ]:
time_series_static_per_day=static_features[static_features.index.isin(mrn_in_cohort)]
time_series_static_per_day

In [ ]:
time_series_static_per_day.to_parquet('Cohort/Time_Series/time_series_per_patient_per_day_static_features.parquet')